# Ten notebook powinien być niemal identyczny jak FINAL - zawiera za to dodatkowe objaśnienia w komentarzach

## Additional data sources

Ze stron poniżej pobrałem dane na temat populacji województw i miast oraz gęstości zaludnienia i powierzchni miast

https://stat.gov.pl/statystyka-regionalna/rankingi-statystyczne/ludnosc-wedlug-wojewodztw/

https://pl.wikipedia.org/wiki/Dane_statystyczne_o_miastach_w_Polsce

## Base configuration

In [ ]:
import helper as h

import pandas as pd
import numpy as np
import time
import re

from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_error as mae

np.random.seed(2018)

price_shift = 50000


## Data

In [ ]:
train = pd.read_hdf('../input/train_property.h5')
test = pd.read_hdf('../input/test_property.h5') 

In [ ]:
stat_miasta = pd.read_csv('../externalData/statystyki-miasta-wiki.csv')
stat_woj = pd.read_csv('../externalData/statystyki-woj.csv')

## FEATURE ENGINEERING

In [ ]:
######################################################
#
# Funkcje pomocnicze
#

#
# konwersja daty na INT na przykład 12 września 2018 -> 201809 
# (pomijam dzień, bo dzięki temu od razu mam kategoryzację na miesiące)
#
def norm_date(value):
    if value is None: return value
    
    months_to_digit = {
        'stycznia': 1,
        'lutego': 2,
        'marca': 3,
        'kwietnia': 4,
        'maja': 5,
        'czerwca': 6,
        'lipca': 7,
        'sierpnia': 8,
        'września': 9,
        'października': 10,
        'listopada': 11,
        'grudnia': 12
    }
    values = value.split(' ')

    day   = int(values[0]) if len(values) == 3 else None
    month = values[-2].lower()
    year  = int(values[-1])

    month = months_to_digit[month]

    return (year*100 + month)

#
# mapowanie tego co znalazłem w 'location' - jeśli istnieje miasto o takiej nazwie
# to przypisuje, w przeciwnym wypadku - 'unknown'
#
def map_city(x):
    if (len(x) == 2):
        if x[1] in miasta_ll_dict:
            return x[1]
        else:
            return 'unknown'
    if (len(x)>2):
        if x[2] in miasta_ll_dict:
            return x[2]
        elif x[1] in miasta_ll_dict:
            return x[1]
        else: 
            return 'unknown'
    
    return 'unknown'

#
# wyciąganie 'visits' z pola 'stats'
#
def get_visit_ads(x):
    if 'visit_ads' in x:
        return np.log( int(x['visit_ads']) + 10 )
    return -1

#
# wyciąganie ceny podanej w słownym opisie (w polu text)
#
def extract_largest_value(x):
    formated = x.lower().replace('m2','').replace(' ','')
    digits = re.split("\D", formated)
    max_value = 0
    if x.lower().find('cena') > 0: # tylko kiedy w tekscie jest 'cena' 
        for i in digits:
            if i.isdigit():
                if len(i) < 7: # ten warunek obcina numery telefonów i inne długie liczbowe identyfikatory
                    if int(i) < 1000000: # w ten sposób odrzucam liczby za duże żeby mogły być użyteczną ceną
                        max_value = max(int(i),max_value)

        if max_value > 100000: # w ten sposób odrzucam za małe ceny (np. ceny za m2)
            return(max_value)
    
    return 0

##########################################################################
#
# Funkcje do kategoryzacji - ten zestaw funkcji dba o to żeby w TRAIN i TEST te same rzeczy dostały 
# tą samą kategorię, przy czym nie muszę łączyć train i test na czas kategoryzacji. Kategorie są 
# tworzone na podstawie danych z train a test jest wypełniany danymi. Jeśli czegoś nie było w train, 
# to test w tym miejscu dostaje 'unkonw'. To pozwala kategoryzować także dane które "przyjdą później"
# bo słowniki kategoryzacji są zapisane w zmiennej. 
#

def categorize_feature(df, feat, feat_cat, indexers, del_feat=True, average_func=np.median, unknown_cat_name='unknown', unknown_cat_val=-1):
    if feat in indexers:
        return categorize_feature_for_test(df, feat, feat_cat, indexers[feat], del_feat=del_feat, unknown_cat_name=unknown_cat_name, unknown_cat_val=unknown_cat_val)
    
    categories_map = categorize_feature_for_train(df, feat, feat_cat, indexers, del_feat=del_feat, unknown_cat_name=unknown_cat_name)
    
def categorize_feature_for_test(df, feat, feat_cat, categories_map, del_feat=True, unknown_cat_name='unknown', unknown_cat_val=-1):
    df[feat_cat] = [categories_map[x] if x in categories_map 
                    else categories_map[unknown_cat_name] if unknown_cat_name in categories_map 
                    else unknown_cat_val for x in df[feat]]
    
    if del_feat:
        del df[feat]
    
def categorize_feature_for_train(df, feat, feat_cat, indexers, del_feat=True, unknown_cat_name='unknown'):
    df.loc[df[feat].isnull(), feat] = unknown_cat_name

    unique_categories = sorted(list(set(df[feat])),key=str)
    
    categories_map = {}

    for i, (cat) in enumerate(unique_categories):
        categories_map[cat] = i

    for cat, ind in categories_map.items():
        df.loc[df[feat] == cat, feat_cat] = ind

    indexers[feat] = categories_map
    
    if del_feat:
        del df[feat]

##########################################################################
#
# Przygotowanie danych
#

def perform_engineering(df, train_indexers=None):
    indexers = train_indexers if train_indexers != None else {}
    
    df['area_float'] = df['area'].map(lambda x: float(re.sub('[^0-9\,\.]','', x).replace(',', '.')))
    # fix values lower than zero in area feature
    df['area_fixed'] = df['area_float'].map(lambda x: x if x > 0 else 60)
    df['floor_int'] = df['floor'].map({'parter':0, '1':1, '2':2, '3':3, -1:-1, '4':4, 
                                       '7':7, '5':5, '10':10, '8':8, '6':6, '9':9,'> 10':11, 
                                       'poddasze':12, 'suterena':-2})
    df['floors_in_int'] = df['floors_in_building'].map(lambda x: 
                                                       int(re.sub('[^0-9\,\.]','', x)) if x != -1 else -1 )
    df['rok_budowy_int'] = df['rok budowy'].map(lambda x: int(re.sub('[^0-9\,\.]','', x)) if x != -1 else -1 )
    df['czynsz_float'] = df['czynsz'].map(lambda x: 
                                          float(re.sub('[^0-9\,\.]','', x).replace(',', '.')) if x != -1 else x )
    
    #
    # LOCATION
    #
    
    # województwa
    df['location_v'] = df['location'].map(lambda x: x[0])
    # drugi poziom lokalizacji
    df['location_c'] = df['location'].map(lambda x: x[0] + ", " + x[1])
    # trzeci poziom lokalizacji
    df['location_cc'] = df['location'].map(
            lambda x: x[0] + ", " + x[1] + ", " + x[2] if len(x)>2 else x[0] + ", " + x[1])  
    # powiaty
    df['location_powiat'] = df['location'].map(lambda x: x[1] if x[1][-1]=='i' else 'unknown')
    # miasta
    df['location_miasto'] = df['location'].map(map_city)
    
    # wypełnienie nowej cechy używając populacji w mieście które jet przypisane
    df['miasto_ludnosc'] = df['location_miasto'].map(miasta_ll_dict)
    # wypełnienie nowej cechy używając gęstości zaludnienia w mieście które jet przypisane
    df['miasto_gestosc'] = df['location_miasto'].map(miasta_gz_dict)
    # wypełnienie nowej cechy używając powierzchni w mieście które jet przypisane
    df['miasto_powierzchnia'] = df['location_miasto'].map(miasta_pow_dict)
    # wypełnienie nowej cechy używając populacji w województwie które jet przypisane
    df['wojewodztwo_ludnosc'] = df['location_v'].map(wojewodztwa_ll)
    
    #
    # tworzenie kategorii dla lokalizacji
    #
    categorize_feature(df, 'location_v', 'location_v_cat', indexers=indexers)
    categorize_feature(df, 'location_c', 'location_c_cat', indexers=indexers)
    categorize_feature(df, 'location_cc', 'location_cc_cat', indexers=indexers)

    categorize_feature(df, 'location_powiat', 'location_powiat_cat', indexers=indexers)
    categorize_feature(df, 'location_miasto', 'location_miasto_cat', indexers=indexers)
    
    #
    # Stats - wyciąganie danych z pola
    #
    
    df['stats_created_at'] = df['stats'].map(lambda x: x['created_at'])
    df['stats_updated_at'] = df['stats'].map(lambda x: x['updated_at'])
    df['stats_visit_ads'] = df['stats'].map(get_visit_ads)
    
    # kategoryzacja danych ze stats
    categorize_feature(df, 'stats_created_at', 'stats_created_at_cat', indexers=indexers)
    categorize_feature(df, 'stats_updated_at', 'stats_updated_at_cat', indexers=indexers)
    
    #
    # pole text - wyciągamy cenę z tekstowego opisu oferty - czasami jest tam 
    # podana cena za nieruchomość. Staramy się znaleźć takie wartości i ich tutaj użyć
    # - patrz funkcja pomocnicza extract_largest_value
    #
    
    df['largest_value'] = df['text'].map(extract_largest_value)
    
    #
    # kategoryzacja pozostałych cech
    #
    categorize_feature(df, 'materiał budynku', 'material_cat', indexers=indexers)
    categorize_feature(df, 'okna', 'okna_cat', indexers=indexers)
    categorize_feature(df, 'stan wykończenia', 'stan_wyk_cat', indexers=indexers)
    categorize_feature(df, 'rodzaj zabudowy', 'rodzaj_zabudowy_cat', indexers=indexers)
    categorize_feature(df, 'ogrzewanie', 'ogrzewanie_cat', indexers=indexers)
    categorize_feature(df, 'forma własności', 'forma_wlasnosci_cat', indexers=indexers)
    
    #
    # czyszczenie dataframe
    #
    columns_to_remove = ['area', 'location', 'data rozpoczęcia', 'stan inwestycji', 'liczba kondygnacji',
                         'floor','floors_in_building','rok budowy', 'czynsz', 'dostępne od',
                         'garaż', 'tarasy', 'ochrona', 'stats', 'text', 'rolety antywłamaniowe', 
                         'kuchenka', 'klimatyzacja', 'plan zagospodarowania:',
                         'telefon', 'telewizja kablowa', 'pom. użytkowe', 'pralka', 'piekarnik',
                         'lodówka', 'ogródek', 'drzwi / okna antywłamaniowe'
                        ]
    for col_to_remove in columns_to_remove:
        if col_to_remove in df: del df[col_to_remove]

    return df, indexers

#
# cechy oparte na cenach są tworzone na podstawie średnich i median cen za metr wyliczonych
# dla danego obszaru (lokalizacje poziomu 1, 2 i 3). Później taka średnia cena jest dla 
# danej nieruchomości mnożona przez powierzchnię nieruchomości i w ten sposób otrzymujemy
# sześć propozycji cen do wykorzystania przy predykcji
#
def price_engineering(df):
    df['lv_mean'] = df['location_v_cat'].map(dict_mean_price_by_LV)
    df['lv_median'] = df['location_v_cat'].map(dict_median_price_by_LV)
    df['lc_mean'] = df['location_c_cat'].map(dict_mean_price_by_LC)
    df['lc_median'] = df['location_c_cat'].map(dict_median_price_by_LC)
    df['lc_mean'] = df['lc_mean'].fillna(df['lv_mean'])
    df['lc_median'] = df['lc_median'].fillna(df['lv_median'])
    df['lcc_mean'] = df['location_cc_cat'].map(dict_mean_price_by_LCC)
    df['lcc_median'] = df['location_cc_cat'].map(dict_median_price_by_LCC)
    df['lcc_mean'] = df['lcc_mean'].fillna(df['lc_mean'])
    df['lcc_median'] = df['lcc_median'].fillna(df['lc_median'])

    df['lv_mean_price_calculated'] = df['lv_mean'] * df['area_fixed']
    df['lv_median_price_calculated'] = df['lv_median'] * df['area_fixed']
    df['lc_mean_price_calculated'] = df['lc_mean'] * df['area_fixed']
    df['lc_median_price_calculated'] = df['lc_median'] * df['area_fixed']
    df['lcc_mean_price_calculated'] = df['lcc_mean'] * df['area_fixed']
    df['lcc_median_price_calculated'] = df['lcc_median'] * df['area_fixed']   
    
    #
    # czyszczenie
    #
    columns_to_remove = ['price_per_meter']
    for col_to_remove in columns_to_remove:
        if col_to_remove in df: del df[col_to_remove]

#########################################
#            
# statystyki dla miast i województw
#
miasta_gz_dict = stat_miasta.groupby('Miasto').agg(np.median)['Gęstość zaludnienia'].to_dict()
miasta_ll_dict = stat_miasta.groupby('Miasto').agg(np.median)['Liczba ludności'].to_dict()
miasta_pow_dict = stat_miasta.groupby('Miasto').agg(np.median)['Powierzchnia'].to_dict()
wojewodztwa_ll = stat_woj.groupby('Województwo').agg(np.median)['Ogółem'].to_dict()
            
## fill NA - niektóre z funkcji powyżej nie lubią pustych danych
train = train.fillna(-1)
test = test.fillna(-1)

## prepare data - pierwsza faza przygotowania danych
%time train, indexers = perform_engineering(train)
%time test, dummy = perform_engineering(test, indexers)

#########################################
#
# tutaj na podstawie danych z train przygotowujemy cenę za metr dla każdego mieszkania 
# a co za tym idzie cenę średnią i medianę za metr w lokalizacjach 1, 2 i 3 poziomu
#
train['price_per_meter'] = train['price'] / train['area_fixed']

dict_mean_price_by_LV = train.groupby('location_v_cat').agg(np.mean)['price_per_meter'].to_dict()
dict_median_price_by_LV = train.groupby('location_v_cat').agg(np.median)['price_per_meter'].to_dict()
dict_mean_price_by_LC = train.groupby('location_c_cat').agg(np.mean)['price_per_meter'].to_dict()
dict_median_price_by_LC = train.groupby('location_c_cat').agg(np.median)['price_per_meter'].to_dict()
dict_mean_price_by_LCC = train.groupby('location_cc_cat').agg(np.mean)['price_per_meter'].to_dict()
dict_median_price_by_LCC = train.groupby('location_cc_cat').agg(np.median)['price_per_meter'].to_dict()

## tutaj obrabiamy train i test dodając cechy oparte na średnich cenach
%time price_engineering(train)
%time price_engineering(test)

## fill NA - jeszcze raz na wszelki wypadek
train = train.fillna(-1)
test = test.fillna(-1)


## Features Selection

In [ ]:
###########################################################################
#
# Przygotowanie cech
#

black_list = ['price', 'id']

bool_features = train.select_dtypes(include=[np.bool]).columns.values.tolist()

cat_feats = [feat for feat in train.columns if 'cat' in feat]
cat_feats = cat_feats + ['rooms', 'is_private', 'floor_int', 'floors_in_int']

numeric_features = train.select_dtypes(include=[np.float64, np.int64, np.int8]).columns.values
numeric_features = [feat for feat in numeric_features if feat not in (black_list + cat_feats) ]

feats = bool_features + numeric_features + cat_feats

feats = [feat for feat in feats if feat not in (black_list)]

#
# przygotowanie zestawu treningowego
#

X = train[ feats ].values
y = train[ 'price' ].values

print("Selected features: ", feats)

# XGB

### Helper functions and configuration

In [ ]:
global fair_constant 

#
# to jest funkcja wewnętrznej metryki dla XGB
#
def fair_obj(y_true, y_pred):
    x = y_pred - y_true
    
    global fair_constant 

    den = abs(x) + fair_constant
    
    grad = fair_constant * x / den
    hess = (fair_constant * fair_constant) / (den * den)
    
    return grad, hess

#
# wartości użyte w tych modelach zostały wyliczone za pomocą hyperopt
# i w niektórych wypadkach nieco poprawione ręcznie podczas testów w zwykłej 
# pętli dla kolejnych wartości które chciałem sprawdzić
#
xgb_params_1 = {
    'objective': fair_obj,
    
    'n_jobs': 4, 
    'max_depth': 8, 
    'n_estimators': 2500, 
    'learning_rate': 0.04, 
    'min_child_weight': 8, 
    'random_state': 4096
}

xgb_params_2 = {
    'objective': fair_obj,
    
    'n_jobs': 4, 
    'max_depth': 14, 
    'n_estimators': 420, 
    'learning_rate': 0.056693922378212164, 
    'min_child_weight': 8, 
    'random_state': 2018
}

xgb_params_s = {
    'objective': fair_obj,
    
    'n_jobs': 4, 
    'max_depth': 8, 
    'n_estimators': 200, 
    'learning_rate': 0.05, 
    'min_child_weight': 8, 
    'random_state': 2018
}

shift = price_shift
fair_constant = 13259.556042072305 # uzyskane używając hyperopt
y_log = np.log(y + shift) # użyłem obliczeń w logarytmach bo wyniki są znacząco lepsze

### Model training

In [ ]:
##
## pierwszy model
##

modelXGB1 = XGBRegressor(**xgb_params_1)
modelXGB1.fit(X, y_log)
    
print('first training finished')    
    
##
## drugi model
##

modelXGB2 = XGBRegressor(**xgb_params_2)
modelXGB2.fit(X, y_log)

print('second training finished')

#
# przygotowanie danych do 'stackowania' i dodanie ich do zbioru treningowego
#
train['XGB_1_results'] = modelXGB1.predict(X)
train['XGB_2_results'] = modelXGB2.predict(X)

stacked_feats = feats + ['XGB_1_results', 'XGB_2_results']

#
# trenowanie stacku używając wcześniejszych danych i nowych cech z dwóch poprzednich modeli
#

X_stacked = train[ stacked_feats ].values
y_stacked = train[ 'price' ].values

y_stacked_log = np.log(y_stacked + shift)

modelXGB_s = XGBRegressor(**xgb_params_s)
modelXGB_s.fit(X_stacked, y_stacked_log)

print('stacked training finished, ready to predict')

In [ ]:
#
# LOG ANSWERS - save the answers to the file
#

#
# to jest funkcja do modyfikacji cen - bazuje na danych które wyciągnęliśmy z pola text
# - jeśli cena przewidziana przez model różni się od tej którą wyciągnęliśm z tekstu
# o mniej niż limit, to zamieniamy ją na tą z tekstu. Nie chcemy skoków o więcej niż limit,
# bo czasem z tekstu uda się wyciągnąć jakieś bzdury.
#
def modPrice(x,limit):
    if x['largest_value'] == 0:
        return x['price']
    if (abs(x['price'] - x['largest_value']) < limit):
        return x['largest_value']
    
    return x['price']

#
# stacking - przygotowujemy predykcje z dwóch pierwszych modeli i ładujemy je do trzeciego
# z którego robimy predykcję
# 
test['XGB_1_results'] = modelXGB1.predict(test[ feats ].values)
test['XGB_2_results'] = modelXGB2.predict(test[ feats ].values)
stacked_feats = feats + ['XGB_1_results', 'XGB_2_results']
y_pred_log = modelXGB_s.predict(test[ stacked_feats ].values)
y_pred = np.exp( y_pred_log ) - shift
print("XGB predictions stacked: ",y_pred)
test['priceXGB_s'] = y_pred

#
# modyfikujemy cenę bazując na cenie z opisu, do limitu 225000
#
test['price'] =  test['priceXGB_s']
test['price_s_mod'] = test.apply(lambda x: modPrice(x,225000), axis=1 )
test['price'] = test['price_s_mod']
test[ ['id', 'price'] ].to_csv('../output/FINAL_' + time.strftime("%Y%m%d_%H%M%S") + '.csv', index=False) 

print("FINISHED")